In [2]:
import h5py
import os.path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gd
from utils.paths import SetupPaths
import matplotlib.patches as  mpatches
from matplotlib.ticker import FormatStrFormatter
from utils.get_summary_data import compile_summary
from utils.vectorCorrection import vectorCorrection as vector


paths = SetupPaths()

In [3]:
plt.show();
plt.rcParams.update({'font.size':20,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})


In [2]:
f = h5py.File(f"{paths.path_subhalos}TNG_99.hdf5",'r')

In [4]:
f['hydro'].keys()

<KeysViewHDF5 ['Group ID', 'Group Mass', 'Group Radius', 'Nsubs', 'Subhalo ID', 'Subhalo Mass', 'Subhalo Max Mass', 'Subhalo Max Mass Snap', 'Subhalo Med Stellar Mass', 'Subhalo Pos', 'Subhalo Stellar Masses', 'Subhalo Vel', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloGasMetallicity', 'SubhaloMassType', 'SubhaloSFR', 'SubhaloSFRinRad']>

In [6]:
np.array(f['hydro']['Subhalo Mass'])[0:10]

array([449.55104651, 158.21931635,  54.95344533,  49.41983106,
        43.27207249,  28.64972518,  26.3520374 ,  11.35363282,
         5.02536597,  10.92833617])

In [16]:
print("num of subhalos in high mass groups:",sum(np.array(f['hydro']['Group Mass'])>100))

num of subhalos in high mass groups: 38350


In [18]:
print("num of subhalos in low mass groups:",sum(np.array(f['hydro']['Group Mass'])<50))

num of subhalos in low mass groups: 44656


In [11]:
np.min(np.array(f['hydro']['Group Mass']))

8.000179416914412

In [25]:
f.close()


In [24]:
f = h5py.File(f"{paths.path_plotdata}counts.hdf5",'r')
f['1000 Realizations']['dwarf'].keys()

<KeysViewHDF5 ['Median All Pairs', 'Median Isolated Primaries', 'Median Major Fraction', 'Median Major Pairs', 'Median Minor Fraction', 'Median Minor Pairs', 'Median Noniso Primaries', 'Median Total Fraction', 'Median Total Primaries', 'Quarts All Pairs', 'Quarts Isolated Primaries', 'Quarts Major Fraction', 'Quarts Major Pairs', 'Quarts Minor Fraction', 'Quarts Minor Pairs', 'Quarts Noniso Primaries', 'Quarts Total Fraction', 'Quarts Total Primaries', 'z']>

In [21]:
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

In [22]:
class EmptyFile(Exception): pass
class SkipRedshift(Exception): pass


## Median virial radius:

In [12]:
pair_data.close()

In [23]:

snapshots = np.arange(0,100,1)
snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
redcutoff = 4.2

redshifts = []  
med_majdw, med_mindw, med_majma, med_minma = [], [], [], []
quart_majdw, quart_mindw, quart_majma, quart_minma = [], [], [], []
med_totdw, med_totma, quart_totdw, quart_totma = [], [], [], []

for snap in snapshots:  
    print(snap)
    
    try:
        pair_path = f"TNG_{snap}_100.hdf5"
        pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")

        if np.size(pair_data) == 0:
            raise EmptyFile

        redshift = pair_data['Header'].attrs['Redshift']

        if redshift > redcutoff :
            raise SkipRedshift

        if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
            raise EmptyFile

        unpair = pair_data["unpaired"]["hydro"]
        unpairStells = np.array(unpair["Sub1 Stellar Mass"])
        unpairGroups = np.array(unpair["Group Mass"])
        unpairReals = np.array(unpair['Realization'])

        pair = pair_data["pairs"]["hydro"]
        priStell = np.array(pair["Sub1 Stellar Mass"])
        secStell = np.array(pair["Sub2 Stellar Mass"])
        pairGroups = np.array(pair["Group Mass"])
        pairRadius = np.array(pair["Group Radius"])
        pairReals = np.array(pair["Realization"])
        seps = np.array(pair["Separation"]) 

            # subset masks for unpaired
        majors = (secStell/priStell > 1/4)
        minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
        allpairs = (majors + minors)
        pair_lowsep = (seps > 10)

        ## dwarfs
        pair_pri_dwarf = get_primmask(priStell, "dwarf")
        pair_group_dwarf = get_groupmask(pairGroups, "dwarf")

            # defining combined masks 
        pair_mask_dwarf = pair_pri_dwarf & pair_group_dwarf & pair_lowsep & allpairs

        ## massive
        pair_pri_massive = get_primmask(priStell, "massive")
        pair_group_massive = get_groupmask(pairGroups, "massive")

            # defining combined masks 
        pair_mask_massive = pair_pri_massive & pair_group_massive & pair_lowsep & allpairs
        
        real_majdw = []
        real_mindw = []
        real_totdw = []
        real_majma = []
        real_minma = []
        real_totma = []
        dat = pairRadius

        for real in np.unique(unpairReals):                  
            # make realization masks
            pair_real = pairReals == real
            
            mask_majdw = pair_real & pair_mask_dwarf & majors
            mask_mindw = pair_real & pair_mask_dwarf & minors
            mask_totdw = pair_real & pair_mask_dwarf
            mask_majma = pair_real & pair_mask_massive & majors
            mask_minma = pair_real & pair_mask_massive & minors
            mask_totma = pair_real & pair_mask_massive

            majdw_xx = np.median( dat[mask_majdw] )
            mindw_xx = np.median( dat[mask_mindw] )
            totdw_xx = np.median( dat[mask_totdw] )
            majma_xx = np.median( dat[mask_majma] )
            minma_xx = np.median( dat[mask_minma] )
            totma_xx = np.median( dat[mask_totma] )

            real_majdw.append( majdw_xx )
            real_mindw.append( mindw_xx )
            real_totdw.append( totdw_xx )
            real_majma.append( majma_xx )
            real_minma.append( minma_xx )
            real_totma.append( totma_xx )
            

        lower, upper = 16,84         
        redshifts.append( redshift )

        med_majdw.append( np.median(real_majdw) )
        med_mindw.append( np.median(real_mindw) )
        med_totdw.append( np.median(real_totdw) )
        med_majma.append( np.median(real_majma) )
        med_minma.append( np.median(real_minma) )
        med_totma.append( np.median(real_totma) )

        quart_majdw.append( np.percentile( real_majdw, [lower,upper] ) )
        quart_mindw.append( np.percentile( real_mindw, [lower,upper] ) )
        quart_totdw.append( np.percentile( real_totdw, [lower,upper] ) )
        quart_majma.append( np.percentile( real_majma, [lower,upper] ) )
        quart_minma.append( np.percentile( real_minma, [lower,upper] ) )
        quart_totma.append( np.percentile( real_totma, [lower,upper] ) )


    except KeyError:
        print(f'skipping {snap} for KeyError. Please debug')
        continue

    except EmptyFile:
        print(f"skipping {snap}, empty file")
        continue

    except SkipRedshift:
        print(f"skipping {snap}, redshift out of range")

count_dictionary = {
        "z": np.array(redshifts),

        "Median Major Dwarf": np.array(med_majdw),
        "Median Minor Dwarf": np.array(med_mindw),
        "Median Total Dwarf": np.array(med_totdw),
        "Median Major Massive": np.array(med_majma),
        "Median Minor Massive": np.array(med_minma),
        "Median Total Massive": np.array(med_totma),


        "Quartile Major Dwarf": np.array(quart_majdw),
        "Quartile Minor Dwarf": np.array(quart_mindw),
        "Quartile Total Dwarf": np.array(quart_totdw),
        "Quartile Major Massive": np.array(quart_majma),
        "Quartile Minor Massive": np.array(quart_minma),
        "Quartile Total High": np.array(quart_totma),
}
pair_data.close()


0
skipping 0, redshift out of range
1
skipping 1, redshift out of range
2
skipping 2, redshift out of range
3
skipping 3, redshift out of range
4
skipping 4, redshift out of range
5
skipping 5, redshift out of range
6
skipping 6, redshift out of range
7
skipping 7, redshift out of range
8
skipping 8, redshift out of range
9
skipping 9, redshift out of range
10
skipping 10, redshift out of range
11
skipping 11, redshift out of range
12
skipping 12, redshift out of range
13
skipping 13, redshift out of range
14
skipping 14, redshift out of range
15
skipping 15, redshift out of range
16
skipping 16, redshift out of range
17
skipping 17, redshift out of range
18
skipping 18, redshift out of range
19
skipping 19, redshift out of range
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [39]:
count_dictionary['z'][29]

0.9972942257819404

In [42]:

zs = count_dictionary['z']
highs = count_dictionary['Median Major Massive']
lows = count_dictionary['Median Major Dwarf']

ind = [-1, 29,13, 5, 1]
for i in ind:
     print("z:%.1f" %zs[i], "--- Rvir high: %.0f" %highs[i], " ---  Rvir low:%.0f" % lows[i])

z:0.0 --- Rvir high: 348  ---  Rvir low:134
z:1.0 --- Rvir high: 206  ---  Rvir low:85
z:2.0 --- Rvir high: 134  ---  Rvir low:59
z:3.0 --- Rvir high: 97  ---  Rvir low:43
z:4.0 --- Rvir high: 76  ---  Rvir low:34


In [24]:
pair_data.close()
print()
for key in count_dictionary.keys():
    print(key, count_dictionary[key][-1])


z 2.220446049250313e-16
Median Major Dwarf 134.33312539122326
Median Minor Dwarf 133.12648663944572
Median Total Dwarf 133.76106440158657
Median Major Massive 348.2726023617415
Median Minor Massive 334.29996996176493
Median Total Massive 342.3335793661022
Quartile Major Dwarf [133.67480674 135.38465227]
Quartile Minor Dwarf [132.18139432 134.02635582]
Quartile Total Dwarf [133.12964494 134.31617662]
Quartile Major Massive [346.8712896  352.19273079]
Quartile Minor Massive [330.86049677 340.16457426]
Quartile Total High [340.1237017  344.75641873]


In [26]:
print()
for key in count_dictionary.keys():
    print(key, count_dictionary[key][13])


z 2.0020281392528516
Median Major Dwarf 58.566022907653604
Median Minor Dwarf 58.062524605926086
Median Total Dwarf 58.334681692794604
Median Major Massive 133.8941734830363
Median Minor Massive 132.479700972061
Median Total Massive 133.09382335229031
Quartile Major Dwarf [58.3152496  58.96213618]
Quartile Minor Dwarf [57.60194404 58.39208222]
Quartile Total Dwarf [58.07144424 58.53253917]
Quartile Major Massive [132.01980614 135.98701564]
Quartile Minor Massive [130.03364153 134.08804603]
Quartile Total High [131.87079393 134.03570327]


In [27]:
print()
for key in count_dictionary.keys():
    print(key, count_dictionary[key][0])


z 4.176834914726472
Median Major Dwarf 33.095830014739406
Median Minor Dwarf 32.96270241062105
Median Total Dwarf 33.06488526016476
Median Major Massive 75.9457791042494
Median Minor Massive 72.01828308054911
Median Total Massive 74.19680816161477
Quartile Major Dwarf [32.76367439 33.42629393]
Quartile Minor Dwarf [32.66060824 33.35511797]
Quartile Total Dwarf [32.76244861 33.21242848]
Quartile Major Massive [71.35500342 79.46013377]
Quartile Minor Massive [70.10878006 77.54458891]
Quartile Total High [70.86938387 76.34350628]


is the group mass a good percent of the combined mass of primary and secondary for major pairs low mass and high mass 

 

In [30]:
pair_data['pairs']['hydro'].keys()

<KeysViewHDF5 ['Comoving Separation', 'Group ID', 'Group Mass', 'Group Nsubs', 'Group Radius', 'Realization', 'RelVel', 'Separation', 'Stellar Mass Ratio', 'Sub1 BHMass', 'Sub1 BHMdot', 'Sub1 GasMetallicity', 'Sub1 ID', 'Sub1 Mass', 'Sub1 MassType', 'Sub1 Pos', 'Sub1 SFR', 'Sub1 SFRinRad', 'Sub1 Stellar Mass', 'Sub1 Vel', 'Sub2 BHMass', 'Sub2 BHMdot', 'Sub2 GasMetallicity', 'Sub2 ID', 'Sub2 Mass', 'Sub2 MassType', 'Sub2 Pos', 'Sub2 SFR', 'Sub2 SFRinRad', 'Sub2 Stellar Mass', 'Sub2 Vel', 'TripleFlag']>

In [31]:

snapshots = np.arange(0,100,1)
snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
redcutoff = 4.2

redshifts = []  
med_majdw, med_mindw, med_majma, med_minma = [], [], [], []
quart_majdw, quart_mindw, quart_majma, quart_minma = [], [], [], []
med_totdw, med_totma, quart_totdw, quart_totma = [], [], [], []

for snap in snapshots:  
    print(snap)
    
    try:
        pair_path = f"TNG_{snap}_100.hdf5"
        pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")

        if np.size(pair_data) == 0:
            raise EmptyFile

        redshift = pair_data['Header'].attrs['Redshift']

        if redshift > redcutoff :
            raise SkipRedshift

        if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
            raise EmptyFile

        unpair = pair_data["unpaired"]["hydro"]
        unpairStells = np.array(unpair["Sub1 Stellar Mass"])
        unpairGroups = np.array(unpair["Group Mass"])
        unpairReals = np.array(unpair['Realization'])

        pair = pair_data["pairs"]["hydro"]
        priStell = np.array(pair["Sub1 Stellar Mass"])
        secStell = np.array(pair["Sub2 Stellar Mass"])
        priSubhalo = np.array(pair["Sub1 Mass"])
        secSubhalo = np.array(pair["Sub2 Mass"])
        pairGroups = np.array(pair["Group Mass"])
        pairRadius = np.array(pair["Group Radius"])
        pairReals = np.array(pair["Realization"])
        seps = np.array(pair["Separation"]) 

            # subset masks for unpaired
        majors = (secStell/priStell > 1/4)
        minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
        allpairs = (majors + minors)
        pair_lowsep = (seps > 10)

        ## dwarfs
        pair_pri_dwarf = get_primmask(priStell, "dwarf")
        pair_group_dwarf = get_groupmask(pairGroups, "dwarf")

            # defining combined masks 
        pair_mask_dwarf = pair_pri_dwarf & pair_group_dwarf & pair_lowsep & allpairs

        ## massive
        pair_pri_massive = get_primmask(priStell, "massive")
        pair_group_massive = get_groupmask(pairGroups, "massive")

            # defining combined masks 
        pair_mask_massive = pair_pri_massive & pair_group_massive & pair_lowsep & allpairs
        
        real_majdw = []
#         real_mindw = []
#         real_totdw = []
        real_majma = []
#         real_minma = []
#         real_totma = []

        for real in np.unique(unpairReals):                  
            # make realization masks
            pair_real = pairReals == real
            
            mask_majdw = pair_real & pair_mask_dwarf & majors
#             mask_mindw = pair_real & pair_mask_dwarf & minors
#             mask_totdw = pair_real & pair_mask_dwarf
            mask_majma = pair_real & pair_mask_massive & majors
#             mask_minma = pair_real & pair_mask_massive & minors
#             mask_totma = pair_real & pair_mask_massive

            majdw_xx = np.median( ( priSubhalo[mask_majdw] +  secSubhalo[mask_majdw]) / pairGroups[mask_majdw] )
#             mindw_xx = np.median( dat[mask_mindw] )
#             totdw_xx = np.median( dat[mask_totdw] )
            majma_xx = np.median( ( priSubhalo[mask_majma] +  secSubhalo[mask_majma]) / pairGroups[mask_majma] )
#             minma_xx = np.median( dat[mask_minma] )
#             totma_xx = np.median( dat[mask_totma] )

            real_majdw.append( majdw_xx )
#             real_mindw.append( mindw_xx )
#             real_totdw.append( totdw_xx )
            real_majma.append( majma_xx )
#             real_minma.append( minma_xx )
#             real_totma.append( totma_xx )
            

        lower, upper = 16,84         
        redshifts.append( redshift )

        med_majdw.append( np.median(real_majdw) )
#         med_mindw.append( np.median(real_mindw) )
#         med_totdw.append( np.median(real_totdw) )
        med_majma.append( np.median(real_majma) )
#         med_minma.append( np.median(real_minma) )
#         med_totma.append( np.median(real_totma) )

#         quart_majdw.append( np.percentile( real_majdw, [lower,upper] ) )
#         quart_mindw.append( np.percentile( real_mindw, [lower,upper] ) )
#         quart_totdw.append( np.percentile( real_totdw, [lower,upper] ) )
#         quart_majma.append( np.percentile( real_majma, [lower,upper] ) )
#         quart_minma.append( np.percentile( real_minma, [lower,upper] ) )
#         quart_totma.append( np.percentile( real_totma, [lower,upper] ) )


    except KeyError:
        print(f'skipping {snap} for KeyError. Please debug')
        continue

    except EmptyFile:
        print(f"skipping {snap}, empty file")
        continue

    except SkipRedshift:
        print(f"skipping {snap}, redshift out of range")

count_dictionary = {
        "z": np.array(redshifts),

        "Median Major Dwarf": np.array(med_majdw),
#         "Median Minor Dwarf": np.array(med_mindw),
#         "Median Total Dwarf": np.array(med_totdw),
        "Median Major Massive": np.array(med_majma),
#         "Median Minor Massive": np.array(med_minma),
#         "Median Total Massive": np.array(med_totma),


#         "Quartile Major Dwarf": np.array(quart_majdw),
#         "Quartile Minor Dwarf": np.array(quart_mindw),
#         "Quartile Total Dwarf": np.array(quart_totdw),
#         "Quartile Major Massive": np.array(quart_majma),
#         "Quartile Minor Massive": np.array(quart_minma),
#         "Quartile Total High": np.array(quart_totma),
}


0
skipping 0, redshift out of range
1
skipping 1, redshift out of range
2
skipping 2, redshift out of range
3
skipping 3, redshift out of range
4
skipping 4, redshift out of range
5
skipping 5, redshift out of range
6
skipping 6, redshift out of range
7
skipping 7, redshift out of range
8
skipping 8, redshift out of range
9
skipping 9, redshift out of range
10
skipping 10, redshift out of range
11
skipping 11, redshift out of range
12
skipping 12, redshift out of range
13
skipping 13, redshift out of range
14
skipping 14, redshift out of range
15
skipping 15, redshift out of range
16
skipping 16, redshift out of range
17
skipping 17, redshift out of range
18
skipping 18, redshift out of range
19
skipping 19, redshift out of range
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [33]:
print()
for key in count_dictionary.keys():
    print(key, count_dictionary[key][-1])
for key in count_dictionary.keys():
    print(key, count_dictionary[key][13])
for key in count_dictionary.keys():
    print(key, count_dictionary[key][0])


z 2.220446049250313e-16
Median Major Dwarf 0.9795863799758089
Median Major Massive 0.9281297178738852
z 2.0020281392528516
Median Major Dwarf 0.926319667059309
Median Major Massive 0.9122509871199274
z 4.176834914726472
Median Major Dwarf 1.0692285764832188
Median Major Massive 1.0187683825995812


In [5]:
fracs.keys()

dict_keys(['Median Major Difference', 'Median Major Dwarf', 'Median Major Massive', 'Median Minor Difference', 'Median Minor Dwarf', 'Median Minor Massive', 'Quartile Major Difference', 'Quartile Major Dwarf', 'Quartile Major Massive', 'Quartile Minor Difference', 'Quartile Minor Dwarf', 'Quartile Minor Massive', 'z'])

In [38]:
for i in range(len(fracs['z'])):
    z = fracs['z'][i]
    medmajor = fracs['Median Major Massive'][i]
    qrtmajor = fracs['Quartile Major Massive'][i]
    medminor = fracs['Median Minor Massive'][i]
    qrtminor = fracs['Quartile Minor Massive'][i]
    
    diffmaj = qrtmajor - medmajor
    diffmin = qrtminor - medminor
    
    toprint = ['z:%.2f'%z, 
               'Major high:%.3f+%.3f%.3f' % (medmajor, diffmaj[1], diffmaj[0]), 
               'Minor high:%.3f+%.3f%.3f' % (medminor, diffmin[1], diffmin[0])]
    
    print(toprint)
    print()

['z:4.18', 'Major high:0.168+0.071-0.067', 'Minor high:0.135+0.086-0.074']

['z:4.01', 'Major high:0.175+0.070-0.060', 'Minor high:0.145+0.083-0.075']

['z:3.71', 'Major high:0.162+0.050-0.047', 'Minor high:0.150+0.063-0.054']

['z:3.49', 'Major high:0.152+0.043-0.038', 'Minor high:0.146+0.058-0.055']

['z:3.28', 'Major high:0.152+0.037-0.040', 'Minor high:0.151+0.047-0.044']

['z:3.01', 'Major high:0.154+0.031-0.031', 'Minor high:0.142+0.041-0.036']

['z:2.90', 'Major high:0.150+0.029-0.030', 'Minor high:0.140+0.038-0.037']

['z:2.73', 'Major high:0.155+0.025-0.026', 'Minor high:0.140+0.032-0.029']

['z:2.58', 'Major high:0.156+0.028-0.026', 'Minor high:0.148+0.029-0.033']

['z:2.44', 'Major high:0.154+0.021-0.025', 'Minor high:0.148+0.030-0.028']

['z:2.32', 'Major high:0.164+0.023-0.024', 'Minor high:0.148+0.027-0.028']

['z:2.21', 'Major high:0.170+0.018-0.021', 'Minor high:0.141+0.026-0.026']

['z:2.10', 'Major high:0.167+0.022-0.021', 'Minor high:0.147+0.024-0.030']

['z:2.00', '

In [36]:
for i in range(len(fracs['z'])):
    z = fracs['z'][i]
    medmajor = fracs['Median Major Dwarf'][i]
    qrtmajor = fracs['Quartile Major Dwarf'][i]
    medminor = fracs['Median Minor Dwarf'][i]
    qrtminor = fracs['Quartile Minor Dwarf'][i]
    
    diffmaj = qrtmajor - medmajor
    diffmin = qrtminor - medminor
    
    toprint = ['z:%.2f'%z, 
               'Major high:%.3f+%.3f%.3f' % (medmajor, diffmaj[1], diffmaj[0]), 
               'Minor high:%.3f+%.3f%.3f' % (medminor, diffmin[1], diffmin[0])]
    
    print(toprint)
    print()

['z:4.18', 'Major high:0.127+0.010-0.011', 'Minor high:0.109+0.010-0.011']

['z:4.01', 'Major high:0.130+0.010-0.010', 'Minor high:0.110+0.011-0.011']

['z:3.71', 'Major high:0.127+0.009-0.009', 'Minor high:0.110+0.010-0.009']

['z:3.49', 'Major high:0.129+0.009-0.010', 'Minor high:0.112+0.008-0.009']

['z:3.28', 'Major high:0.128+0.007-0.008', 'Minor high:0.111+0.008-0.009']

['z:3.01', 'Major high:0.126+0.008-0.007', 'Minor high:0.111+0.008-0.008']

['z:2.90', 'Major high:0.126+0.007-0.007', 'Minor high:0.110+0.007-0.007']

['z:2.73', 'Major high:0.126+0.007-0.008', 'Minor high:0.110+0.007-0.007']

['z:2.58', 'Major high:0.124+0.006-0.007', 'Minor high:0.108+0.007-0.008']

['z:2.44', 'Major high:0.123+0.006-0.007', 'Minor high:0.106+0.006-0.007']

['z:2.32', 'Major high:0.123+0.007-0.006', 'Minor high:0.105+0.006-0.007']

['z:2.21', 'Major high:0.122+0.006-0.006', 'Minor high:0.104+0.006-0.007']

['z:2.10', 'Major high:0.118+0.006-0.005', 'Minor high:0.102+0.006-0.007']

['z:2.00', '

In [37]:
for i in range(len(fracs['z'])):
    z = fracs['z'][i]
    medmajor = fracs['Median Major Difference'][i]
    qrtmajor = fracs['Quartile Major Difference'][i]
    medminor = fracs['Median Minor Difference'][i]
    qrtminor = fracs['Quartile Minor Difference'][i]
    
    diffmaj = qrtmajor - medmajor
    diffmin = qrtminor - medminor
    
    toprint = ['z:%.2f'%z, 
               'Major high:%.3f+%.3f%.3f' % (medmajor, diffmaj[1], diffmaj[0]), 
               'Minor high:%.3f+%.3f%.3f' % (medminor, diffmin[1], diffmin[0])]
    
    print(toprint)
    print()

['z:4.18', 'Major high:0.042+0.072-0.069', 'Minor high:0.008+0.085-0.082']

['z:4.01', 'Major high:0.044+0.070-0.062', 'Minor high:0.015+0.088-0.073']

['z:3.71', 'Major high:0.035+0.049-0.048', 'Minor high:0.024+0.062-0.053']

['z:3.49', 'Major high:0.023+0.047-0.040', 'Minor high:0.017+0.058-0.056']

['z:3.28', 'Major high:0.023+0.038-0.039', 'Minor high:0.022+0.048-0.043']

['z:3.01', 'Major high:0.028+0.033-0.033', 'Minor high:0.017+0.040-0.038']

['z:2.90', 'Major high:0.025+0.032-0.032', 'Minor high:0.014+0.040-0.036']

['z:2.73', 'Major high:0.029+0.026-0.026', 'Minor high:0.014+0.034-0.031']

['z:2.58', 'Major high:0.032+0.029-0.027', 'Minor high:0.024+0.029-0.033']

['z:2.44', 'Major high:0.031+0.022-0.024', 'Minor high:0.025+0.029-0.029']

['z:2.32', 'Major high:0.041+0.025-0.023', 'Minor high:0.025+0.027-0.029']

['z:2.21', 'Major high:0.048+0.019-0.020', 'Minor high:0.019+0.029-0.027']

['z:2.10', 'Major high:0.049+0.022-0.023', 'Minor high:0.029+0.024-0.026']

['z:2.00', '

In [25]:
print(fracs['Median Major Massive'][10])
print(fracs['Quartile Major Massive'][10])

0.16414686825053995
[0.14023732 0.18675353]


In [27]:
fracs['Quartile Major Massive'][10]-fracs['Median Major Massive'][10]

array([-0.02390954,  0.02260666])

In [28]:
fracs['Quartile Major Massive'][10][0] - fracs['Median Major Massive'][10]

-0.023909543547195822

# counts of massive 

In [26]:
f = h5py.File(f"{paths.path_plotdata}counts.hdf5",'r')

counts = {}
for key,val in f[f"1000 Realizations"]['massive'].items():
    counts[key] = np.array(val)

f.close()

In [28]:
for i in range(len(counts['z'])):
    print('za:' counts['z'][i], "Median counts["Median"]

SyntaxError: invalid syntax (<ipython-input-28-188155c021cc>, line 2)

In [14]:
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

In [19]:
class EmptyFile(Exception): pass
size="massive"

snapshots = np.arange(0,100,1)
snapshots = np.delete(snapshots, np.where(snapshots==48)[0])

redshifts = []
medone, medtwo, medtot, medmaj, medmin, medpair, medmajfrac, medminfrac, medtotfrac = [], [], [], [], [], [], [], [], []
quartone, quarttwo, quarttot, quartmaj, quartmin, quartpair, quartmajfrac, quartminfrac, quarttotfrac = [], [], [], [], [], [], [], [], []

for snap in snapshots:  
    try:
        pair_path = f"TNG_{snap}_100.hdf5"
        pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")

        if np.size(pair_data) == 0:
            raise EmptyFile

        if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
            raise EmptyFile        

        redshift = pair_data['Header'].attrs['Redshift']

        unpair = pair_data["unpaired"]["hydro"]
        unpairStells = np.array(unpair["Sub1 Stellar Mass"])
        unpairGroups = np.array(unpair["Group Mass"])
        unpairReals = np.array(unpair['Realization'])

        pair = pair_data["pairs"]["hydro"]
        priStell = np.array(pair["Sub1 Stellar Mass"])
        secStell = np.array(pair["Sub2 Stellar Mass"])
        pairGroups = np.array(pair["Group Mass"])
        pairReals = np.array(pair["Realization"])
        seps = np.array(pair["Separation"]) 

        # subset masks for unpaired
        unpair_pri = get_primmask(unpairStells, size)
        unpair_group = get_groupmask(unpairGroups, size)                

        pair_pri = get_primmask(priStell, size)
        pair_group = get_groupmask(pairGroups, size)


        # defining combined masks 
        unpair_mask = unpair_pri & unpair_group
        primary_mask = pair_pri & pair_group
              
        pair_mask = pair_pri & pair_group & pair_lowsep & allpairs
                                                           
        for real in np.unique(unpairReals):                  
            # make realization masks
            unpair_real = unpairReals == real
            pair_real = pairReals == real

            # make count values for single realization
            numone = np.count_nonzero(unpair_mask & unpair_real)
            numtwo = np.count_nonzero(primary_mask & pair_real)
            numtot = numone + numtwo
         
            if numtot == 0:
                continue

            # collect count vals for all reals
            totalprims.append(numtot)
         
            # create arrays of medians and quartiles~ 
            lower, upper = 0.5, 99.5                
            redshifts.append( redshift )
            medtot.append(np.median( totalprims ))
            quarttot.append( np.percentile( totalprims, [lower, upper]))


        # create arrays of medians and quartiles~ 

        print(redshift,numtot)
        
    except EmptyFile:
        continue
            


9.996590466186333 0
9.388771271940549 0
9.00233985416247 0
8.449476294368743 0
8.012172948865935 0
7.5951071498715965 0
7.23627606616736 0
7.005417045544533 0
6.491597745667503 0
6.0107573988449 3
5.846613747881867 3
5.5297658079491026 6
5.227580973127337 19
4.995933468164624 28
4.664517702470927 45
4.428033736605549 71
4.176834914726472 95
4.0079451114652676 123
3.7087742646422353 206
3.4908613692606485 272
3.2830330579565246 329
3.008131071630377 472
2.8957850057274284 543
2.7331426173187188 641
2.5772902716018935 742
2.4442257045541464 833
2.3161107439568918 944
2.207925472383703 984
2.1032696525957713 1041
2.0020281392528516 1114
1.9040895435327672 1173
1.822689252620354 1235
1.7435705743308647 1280
1.6666695561144653 1358
1.6042345220731056 1411
1.5312390291576135 1447
1.4955121664955557 1481
1.4140982203725216 1546
1.3575766674029972 1582
1.3023784599059653 1621
1.2484726142451428 1652
1.2062580807810006 1700
1.1546027123602154 1715
1.1141505637653806 1749
1.035510445664141 1756


In [7]:

snapshots = np.arange(0,100,1)
snapshots = np.delete(snapshots, np.where(snapshots==48)[0])

redshifts = []
medone, medtwo, medtot, medmaj, medmin, medpair, medmajfrac, medminfrac, medtotfrac = [], [], [], [], [], [], [], [], []
quartone, quarttwo, quarttot, quartmaj, quartmin, quartpair, quartmajfrac, quartminfrac, quarttotfrac = [], [], [], [], [], [], [], [], []

for snap in snapshots:  
    singleprims, doubleprims, totalprims = [], [], []
    majorpairs, minorpairs, totalpairs = [], [], []
    majfrac, minfrac, totfrac = [], [], []

    try:
        pair_path = f"TNG_{snap}_100.hdf5"
        pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")

        if np.size(pair_data) == 0:
            raise EmptyFile

        redshift = pair_data['Header'].attrs['Redshift']


        if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
            raise EmptyFile

        unpair = pair_data["unpaired"]["hydro"]
        unpairStells = np.array(unpair["Sub1 Stellar Mass"])
        unpairGroups = np.array(unpair["Group Mass"])
        unpairReals = np.array(unpair['Realization'])

        pair = pair_data["pairs"]["hydro"]
        priStell = np.array(pair["Sub1 Stellar Mass"])
        secStell = np.array(pair["Sub2 Stellar Mass"])
        pairGroups = np.array(pair["Group Mass"])
        pairReals = np.array(pair["Realization"])
        seps = np.array(pair["Separation"]) 

        # subset masks for unpaired
        unpair_pri = get_primmask(unpairStells, size)
        unpair_group = get_groupmask(unpairGroups, size)                

        pair_pri = get_primmask(priStell, size)
        pair_group = get_groupmask(pairGroups, size)

        majors = (secStell/priStell > 1/4)
        minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
        allpairs = (majors + minors)
        pair_lowsep = (seps > 10)

        # defining combined masks 
        unpair_mask = unpair_pri & unpair_group
        primary_mask = pair_pri & pair_group
        pair_mask = pair_pri & pair_group & pair_lowsep & allpairs

        for real in np.unique(unpairReals):                  
            # make realization masks
            unpair_real = unpairReals == real
            pair_real = pairReals == real

            # make count values for single realization
            numone = np.count_nonzero(unpair_mask & unpair_real)
            numtwo = np.count_nonzero(primary_mask & pair_real)
            numtot = numone + numtwo


            if numtot == 0:
                continue
            totalprims.append(numtot)

        # create arrays of medians and quartiles~ 

        redshifts.append( redshift )
        medtot.append(np.median( totalprims ))


SyntaxError: unexpected EOF while parsing (<ipython-input-7-b09fa0a0bda5>, line 76)

## counts 

In [40]:
counts.keys()

dict_keys(['Median All Pairs', 'Median Isolated Primaries', 'Median Major Fraction', 'Median Major Pairs', 'Median Minor Fraction', 'Median Minor Pairs', 'Median Noniso Primaries', 'Median Total Fraction', 'Median Total Primaries', 'Quarts All Pairs', 'Quarts Isolated Primaries', 'Quarts Major Fraction', 'Quarts Major Pairs', 'Quarts Minor Fraction', 'Quarts Minor Pairs', 'Quarts Noniso Primaries', 'Quarts Total Fraction', 'Quarts Total Primaries', 'z'])

In [55]:
f = h5py.File(f"{paths.path_plotdata}counts.hdf5",'r')

counts = {}
for key,val in f[f"1000 Realizations"]['massive'].items():
    counts[key] = np.array(val)

f.close()

for i in range(len(counts['z'])):
    z = counts['z'][i]
    med = counts['Median Total Fraction'][i]
    qrt = counts['Quarts Total Fraction'][i]
    
    diff = qrt - med
    
    print("z : %.2f" % counts['z'][i],"total pairs : %.3f+%.3f%.3f" % (med, diff[1], diff[0]))#",counts['Median Total Primaries'][i])

# for i in range(len(fracs['z'])):
#     z = fracs['z'][i]
#     medmajor = fracs['Median Major Difference'][i]
#     qrtmajor = fracs['Quartile Major Difference'][i]
#     medminor = fracs['Median Minor Difference'][i]
#     qrtminor = fracs['Quartile Minor Difference'][i]
    
#     diffmaj = qrtmajor - medmajor
#     diffmin = qrtminor - medminor
    
#     toprint = ['z:%.2f'%z, 
#                'Major high:%.3f+%.3f%.3f' % (medmajor, diffmaj[1], diffmaj[0]), 
#                'Minor high:%.3f+%.3f%.3f' % (medminor, diffmin[1], diffmin[0])]
    
#     print(toprint)
#     print() 

z : 4.18 total pairs : 0.306+0.078-0.075
z : 4.01 total pairs : 0.320+0.075-0.068
z : 3.71 total pairs : 0.313+0.062-0.065
z : 3.49 total pairs : 0.298+0.053-0.054
z : 3.28 total pairs : 0.302+0.050-0.051
z : 3.01 total pairs : 0.296+0.040-0.039
z : 2.90 total pairs : 0.291+0.035-0.036
z : 2.73 total pairs : 0.296+0.033-0.031
z : 2.58 total pairs : 0.303+0.030-0.032
z : 2.44 total pairs : 0.303+0.027-0.028
z : 2.32 total pairs : 0.311+0.029-0.028
z : 2.21 total pairs : 0.311+0.025-0.026
z : 2.10 total pairs : 0.315+0.024-0.025
z : 2.00 total pairs : 0.316+0.023-0.024
z : 1.90 total pairs : 0.319+0.023-0.021
z : 1.82 total pairs : 0.318+0.022-0.022
z : 1.74 total pairs : 0.321+0.020-0.021
z : 1.67 total pairs : 0.323+0.020-0.020
z : 1.60 total pairs : 0.329+0.022-0.020
z : 1.53 total pairs : 0.324+0.018-0.021
z : 1.50 total pairs : 0.318+0.019-0.024
z : 1.41 total pairs : 0.318+0.021-0.021
z : 1.36 total pairs : 0.318+0.019-0.016
z : 1.30 total pairs : 0.316+0.020-0.020
z : 1.25 total p

In [34]:
counts['Median Total Primaries']

array([  97.,  128.,  203.,  266.,  334.,  468.,  543.,  639.,  743.,
        842.,  923.,  997., 1049., 1124., 1184., 1240., 1284., 1354.,
       1408., 1458., 1479., 1556., 1574., 1606., 1646., 1682., 1715.,
       1741., 1757., 1778., 1826., 1830., 1839., 1863., 1883., 1886.,
       1886., 1888., 1873., 1875., 1885., 1890., 1883., 1888., 1894.,
       1899., 1901., 1896., 1889., 1892., 1887., 1880., 1880., 1879.,
       1862., 1858., 1853., 1839., 1831., 1834., 1850., 1841., 1848.,
       1858., 1856., 1848., 1860., 1857., 1839., 1821., 1816., 1813.,
       1822., 1811., 1812., 1806., 1794., 1808., 1802.])

## stellar mass 

In [100]:
for size in ["dwarf","massive"]:
    for z in [0,1,2,3,4]:

        f = h5py.File(f"{paths.path_plotdata}smr.hdf5",'r')
        # f['1000 Realizations']['z=0']['dwarf'].keys()
        majj = np.array(f['1000 Realizations'][f'z={z}'][size]['major'])
        minn = np.array(f['1000 Realizations'][f'z={z}'][size]['minor'])
        f.close()

        bins_d = np.arange(0,1.05,0.05)
        bins, edges = np.histogram(majj,bins=bins_d, weights=np.ones(len(majj))/ (len(majj) + len(minn)))
        bins2, edges2 = np.histogram(minn,bins=bins_d, weights=np.ones(len(minn))/ (len(majj) + len(minn)))

        print("LOW Mass: There are %0.1f more pairs with 1:4 than 1:1" % (bins[5]/bins[-1]))

#         print("LOW Mass: There are %0.1f more pairs with 1:10 than 1:4" % (bins2[2]/bins2[4]))



LOW Mass: There are 3.8 more pairs with 1:4 than 1:1
LOW Mass: There are 4.3 more pairs with 1:4 than 1:1
LOW Mass: There are 4.5 more pairs with 1:4 than 1:1
LOW Mass: There are 4.8 more pairs with 1:4 than 1:1
LOW Mass: There are 4.5 more pairs with 1:4 than 1:1
LOW Mass: There are 4.2 more pairs with 1:4 than 1:1
LOW Mass: There are 3.3 more pairs with 1:4 than 1:1
LOW Mass: There are 4.4 more pairs with 1:4 than 1:1
LOW Mass: There are 4.7 more pairs with 1:4 than 1:1
LOW Mass: There are 4.3 more pairs with 1:4 than 1:1


In [106]:
for size in ["dwarf","massive"]:
    for z in [0,1,2,3,4]:

        f = h5py.File(f"{paths.path_plotdata}smr.hdf5",'r')
        # f['1000 Realizations']['z=0']['dwarf'].keys()
        majj = np.array(f['1000 Realizations'][f'z={z}'][size]['major'])
        minn = np.array(f['1000 Realizations'][f'z={z}'][size]['minor'])
        f.close()

        bins_d = np.arange(0,1.05,0.05)
        bins, edges = np.histogram(majj,bins=bins_d, weights=np.ones(len(majj))/ (len(majj) + len(minn)))
        bins2, edges2 = np.histogram(minn,bins=bins_d, weights=np.ones(len(minn))/ (len(majj) + len(minn)))

        print("LOW Mass: %.4f" % (bins[-1]))

#         print("LOW Mass: There are %0.1f more pairs with 1:10 than 1:4" % (bins2[2]/bins2[4]))



LOW Mass: 0.0204
LOW Mass: 0.0182
LOW Mass: 0.0178
LOW Mass: 0.0170
LOW Mass: 0.0177
LOW Mass: 0.0200
LOW Mass: 0.0228
LOW Mass: 0.0179
LOW Mass: 0.0171
LOW Mass: 0.0186


In [81]:
print(bins, edges)

[0.         0.         0.         0.         0.         0.08049979
 0.06569347 0.05514073 0.04732051 0.04100824 0.03646108 0.03300955
 0.02976126 0.02693549 0.02488556 0.02266515 0.02136628 0.01991588
 0.01870004 0.01773263] [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


In [93]:
bins[5]/bins[-1]

4.539643414672671

In [82]:
print(bins2, edges2)

[0.         0.         0.21404011 0.14121557 0.10364866 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ] [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


In [85]:
bins2[2]/bins2[4]

2.065054364250801

## Massive pair fracs from z=2-4

In [8]:
f = h5py.File(f"{paths.path_plotdata}pairfrac.hdf5",'r')
fracs={}
for key, val in f['1000 Realizations'].items():
    fracs[key] = np.array(val)
f.close()

In [19]:
print(max(fracs['Median Major Massive'][0:30]), min(fracs['Median Major Massive'][0:30]))

0.1794326241134752 0.1503623188405797


In [20]:
print(max(fracs['Median Minor Massive'][0:30]), min(fracs['Median Minor Massive'][0:30]))

0.15060240963855423 0.1348314606741573


In [16]:
for i in range(len(fracs['z']))[0:30]:
    z = fracs['z'][i]
    medmajor = fracs['Median Major Massive'][i]
#     qrtmajor = fracs['Quartile Major Massive'][i]
    medminor = fracs['Median Minor Massive'][i]
    
    print(i,z, medmajor, medminor)
#     qrtminor = fracs['Quartile Minor Massive'][i]
    
#     diffmaj = qrtmajor - medmajor
#     diffmin = qrtminor - medminor
    
#     toprint = ['z:%.2f'%z, 
#                'Major high:%.3f+%.3f%.3f' % (medmajor, diffmaj[1], diffmaj[0]), 
#                'Minor high:%.3f+%.3f%.3f' % (medminor, diffmin[1], diffmin[0])]
    
#     print(toprint)
#     print()

0 4.176834914726472 0.16842105263157894 0.1348314606741573
1 4.0079451114652676 0.1746031746031746 0.1450381679389313
2 3.7087742646422353 0.16176470588235295 0.15048543689320387
3 3.4908613692606485 0.15185185185185185 0.14615384615384616
4 3.2830330579565246 0.15170278637770898 0.15060240963855423
5 3.008131071630377 0.1535181236673774 0.14224137931034483
6 2.8957850057274284 0.1503623188405797 0.13996316758747698
7 2.7331426173187188 0.15538461538461537 0.14021571648690292
8 2.5772902716018935 0.15561569688768606 0.14769647696476965
9 2.4442257045541464 0.15447154471544716 0.148014440433213
10 2.3161107439568918 0.16414686825053995 0.14778856526429343
11 2.207925472383703 0.16981132075471697 0.14140386571719227
12 2.1032696525957713 0.16682554814108674 0.14744801512287334
13 2.0020281392528516 0.16666666666666666 0.14942528735632185
14 1.9040895435327672 0.17114093959731544 0.14786967418546365
15 1.822689252620354 0.17182410423452768 0.14653784219001612
16 1.7435705743308647 0.17421

## low mass sep cut vals

In [48]:
f = h5py.File(f"{paths.path_plotdata}pairfrac_sepcut.hdf5",'r')
fracs={}
for key, val in f['1000 Realizations']['Sepcut 150kpc'].items():
    fracs[key] = np.array(val)
f.close()

In [50]:
fracs.keys()

dict_keys(['Frac Major Dwarf Recovered', 'Frac Major Massive Recovered', 'Frac Minor Dwarf Recovered', 'Frac Minor Massive Recovered', 'Median Major Dwarf', 'Median Major Massive', 'Median Minor Dwarf', 'Median Minor Massive', 'Quartile Major Dwarf', 'Quartile Major Dwarf Recovered', 'Quartile Major Massive', 'Quartile Major Massive Recovered', 'Quartile Minor Dwarf', 'Quartile Minor Dwarf Recovered', 'Quartile Minor Massive', 'Quartile Minor Massive Recovered', 'z'])

In [57]:
for i in range(len(fracs['z'])):
    print("%.4f"%fracs['z'][i])

4.1768
4.0079
3.7088
3.4909
3.2830
3.0081
2.8958
2.7331
2.5773
2.4442
2.3161
2.2079
2.1033
2.0020
1.9041
1.8227
1.7436
1.6667
1.6042
1.5312
1.4955
1.4141
1.3576
1.3024
1.2485
1.2063
1.1546
1.1142
1.0355
0.9973
0.9505
0.9230
0.8869
0.8515
0.8167
0.7911
0.7574
0.7326
0.7001
0.6761
0.6446
0.6214
0.5985
0.5760
0.5464
0.5246
0.5030
0.4818
0.4609
0.4403
0.4200
0.3999
0.3802
0.3607
0.3479
0.3288
0.3101
0.2977
0.2734
0.2613
0.2435
0.2260
0.2144
0.1973
0.1804
0.1693
0.1527
0.1419
0.1258
0.1099
0.0994
0.0839
0.0737
0.0585
0.0485
0.0337
0.0240
0.0095
0.0000


In [58]:
# fracs['Median Major Dwarf']
fracs['Median Minor Dwarf']

array([0.10826155, 0.10967291, 0.10897531, 0.11116972, 0.10987318,
       0.10955614, 0.10911247, 0.10822591, 0.10642919, 0.10458548,
       0.10294118, 0.10125406, 0.09856712, 0.09730329, 0.09501872,
       0.09281479, 0.09086444, 0.08954812, 0.08684695, 0.08451274,
       0.08301559, 0.08042771, 0.07801137, 0.0759552 , 0.07397224,
       0.07199372, 0.06983295, 0.06744657, 0.0638673 , 0.06256504,
       0.06012246, 0.0593752 , 0.05842303, 0.05727214, 0.05526893,
       0.05391395, 0.05286029, 0.05112792, 0.04925981, 0.0489515 ,
       0.04776331, 0.04642004, 0.04498898, 0.04396317, 0.04254769,
       0.04184563, 0.04032469, 0.03944864, 0.03848942, 0.03741722,
       0.03654064, 0.03584038, 0.03519296, 0.03448973, 0.0336379 ,
       0.03300892, 0.03184059, 0.0313512 , 0.03035133, 0.02918942,
       0.02869804, 0.02822695, 0.02806468, 0.02706399, 0.02637681,
       0.02600058, 0.02538445, 0.02476895, 0.02401204, 0.02344345,
       0.0228833 , 0.02264268, 0.02238572, 0.02195277, 0.02187

In [52]:
fracs['Quartile Major Dwarf']

array([[0.11467083, 0.13599526],
       [0.11895197, 0.13880071],
       [0.11636202, 0.13423221],
       [0.11812574, 0.13670853],
       [0.11789622, 0.13338105],
       [0.1154533 , 0.13117653],
       [0.11604062, 0.13048213],
       [0.11516776, 0.13005191],
       [0.11395591, 0.12684953],
       [0.11275157, 0.12577045],
       [0.11135109, 0.12471874],
       [0.11031008, 0.12190035],
       [0.1071618 , 0.11856569],
       [0.10506497, 0.11640367],
       [0.10229959, 0.11293812],
       [0.09779292, 0.10881935],
       [0.09618075, 0.1061021 ],
       [0.09403354, 0.10455266],
       [0.09053048, 0.10096252],
       [0.0875489 , 0.09779009],
       [0.08693612, 0.09608446],
       [0.0831733 , 0.09252386],
       [0.07970871, 0.08888889],
       [0.07687763, 0.08625567],
       [0.07502785, 0.08342581],
       [0.07343413, 0.08225108],
       [0.07147541, 0.08012084],
       [0.06884557, 0.07726558],
       [0.06438105, 0.07190265],
       [0.06327382, 0.07091003],
       [0.

In [53]:
np.array([0.02033026, 0.02545757])-0.02307756

array([-0.0027473 ,  0.00238001])